# Continuous Control Project Report
-----

_2/15/2021_

Within the Udacity Deep Reinforcement Learning nano-degree program, Continuous Control is the second major project for students to build on their own.  The project is to build and train an agent using any policy-based techniques to control the Reacher two-jointed robot arm in the Unity ML Agents environment.  The object is to move the arm in such a way that its end point is always in the vicinity of the environment's moving target. The target is a sphere that randomly moves in a circular pattern in a plane parallel to the base table.  It moves either clockwise or counter-clockwise, and varies its speed.  Sometimes it is essentially stationary.  This is not an episodic task, so we arbitrarily limit trajectory length to a certain number of time steps and call it an episode.

The top level code for this project is in the _cont-ctrl-project.ipynb_ Jupyter notebook, which uses code in separate Python source files in the same directory.  The project goals & structure are further described [here](README.md).

## Algorithm

To solve this project I chose to use the Deep Deterministic Policy Gradient (DDPG) algorithm,
[described in this paper](https://arxiv.org/abs/1509.02971).  I chose to train it on a single agent.  DDPG involves several tunable parameters in addition to all the parameters required for the two neural networks it uses.  Below I describe work I did to tune some of them.

### Neural Networks

DDPG uses four neural nets, the local actor, the local critic, and separate target nets for each of these.  The local and target versions of each need to have identical structure; their differences lie in the values of the weights, and need to be slightly different.  While there is no rule that the actor and critic need to have similar architectures, I did not see a reason to make them significantly different.  There are subtle differences, in that the actor outputs a 4-element vector, while the critic only outputs a single value.  Also, the critic net needs to take in the actor's outputs and merge them into the rest of its layer processing, so its output layer is a bit larger than the actor's.  The structure I used was:

- Actor networks
    - 33 state inputs
    - FC layer 1 has 400 nodes and relu activation
    - FC layer 2 has 256 nodes and relu activation
    - FC layer 3 has 4 nodes and tanh activation (output is the action vector)

- Critic networks
    - 33 state inputs
    - FC layer 1 has 400 nodes and relu activation
    - FC layer 2 has 256 nodes and relu activation (the 4 actor outputs are concatenated to the output of layer 1 so this layer has 404 input values)
    - FC layer 3 has 1 node and linear activation (output is the Q value of the (state, action) pair)
    
All networks use an Adam optimizer to propagate gradients.

### Algorithm Enhancements

I added some enhancements to the DDPG algorithm as reported in the original paper.
- Gradient clipping was added to both the actor and critic local updates after calculating the loss but before propagating this loss through the network.  This helps to stabilize learning where changes could otherwise be excessive on some nodes.
- Noise is added to the actor's action output, but the noise is gradually diminished with each time step.
- Training rate controls allow more flexibility in the offline learning.  Instead of doing a learning pass after every action time step, which can move the network weights too rapidly, learning is restricted to once every N time steps.  Further, once that Nth time step occurs, it is advantageous to do multiple training iterations at once, all looking at the same replay buffer.

## Parameters Used

Parameters used in this solution are as follows.

- Replay buffer size:  1e6 experiences
- Future discount (gamma):  0.99
- Soft update rate (tau):  0.001
- Learning rate, actor:  0.001
- Learning rate, critic:  0.001
- L2 weight decay:  1e-5
- Minibatch size:  512 experiences
- Noise decay rate:  0.99999
- Learning every:  25 time steps
- Learning iterations:  10
- Max time steps per episode:  1000

I found it difficult to determine the suitable combination of parameters show above that would result in a converged learning solution.  Major ones I experimented with are:
- **Number of time steps per "episode".** I found a solution that converged rapidly (223 episodes) with 1000 time steps each. I then tried training with only 400 time steps.  This training performance also looked good, but took a long time. It did not converge within 1000 episodes, but hit an average score of 15+ at that point.  Visual observation of the Unity environment at this point looked like it was performing really well, however; even when the arm end is not in the sphere it is not far away, and normally moving in the right direction.  With nominally 0.04 reward points per time step in the target sphere, the theoretical max score for 400 time steps is 16, so my solution was nearly perfect.
- **Minibatch size.** This has a major impact on training convergence.  It seems to be intimately tied to other "training rate" parameters (explained below).  In summary, it appears a large batch size is better, as it spreads the learning over a wide variety of experiences, averaging the gradient calculations over all of them.  With small batches it seems the gradients are too likely to go in the wrong direction, making training unstable.  I imagine this is exacerbated by the fact that we are only looking at one time step at a time, not any history of motion, so any given time step provides too narrow a picture of the world to learn much from.  My soltion used a batch size of 512 experiences, although I found that 256 could be made to work also.
- **Training rate.**  This category encompasses two related parameters that control when and how much batch learning occurs.  The first parameter I call _learn_every_ , which restricts learning to only happen every N time steps.  Thus, the replay buffer gets built up with several new experiences before the next learning session is invoked.  Once it is time to learn, the second parameter, _learn_iterations_ , specifies that M iterations of learning (M minibatches) are to occur before the next time step is run.  I found that 25 and 10 worked well for these parameters, respectively.  So we collect 25 new experiences (time steps), then stop and do 10 training iterations (a full minibatch per iteration).  In this way, the algorithm is allowed to "cram" the new learning material in intense bursts, with each burst more likely to see experiences from similar sets of network weights, so the gradients calculated are more meaningful and probably less harsh.  I was able to reduce the _learn_every_ value somewhat, but training was slower.


## Results

I spent several hours poking at various combinations of hyperparameters, NN designs and combinations of the three algorithm enhancements noted above before I saw any significant training.  Finally finding a combination that works felt like shooting in the dark, but perseverance paid off.  Once I found that sweet spot I explored various combinations nearby to get a sense of the sensitivities to various parameters.  The best training performance was with the values shown above and captured in the associated code.  Here is its training history.  ![plot](Learning_plot_A.png)

## Future Considerations

I suspect training could be improved if the reward were related to the distance from the target (e.g. proportional to 1/radius^3), rather than making it a step function.  As is, the arm tends to bounce around within that large target sphere, which doesn't allow it to learn a lot of precision motion.  But doing this would require modifying the environment model.

I plan to keep this project around for further experimentation with the hyperparameters.  As I move into work projects that require continuous control I will undoubtedly have similar tuning problems and this will give me some reference for how they affect the learning ability.